As the results with Conifer are not looking very promising, let's give CNVkit another shot.

The thing about CNVkit, along with many other WES CNV tools, is that it depends on a reference. It does however have an option of constructing the reference with zero samples. So, here are a few options we can test:
 * no reference
 * siblings are reference
 * all other family members are reference
 * all controls in study are reference

Let's then start with that first option, which is equivalent to the "power user" option in their manual:

In [ ]:
cd ~/data/cnv/cnvkit
module load cnvkit

mkdir BAM
while read s; do
    ln -s /data/NCR_SBRB/simplex/BAM/${s}/${s}_sorted_RG_markduplicate_recalibrated.bam BAM/${s}.bam;
    ln -s /data/NCR_SBRB/simplex/BAM/${s}/${s}_sorted_RG_markduplicate_recalibrated.bai BAM/${s}.bai;
done < ../conifer84/samples.txt

In [ ]:
exome_targets='/data/NCR_SBRB/simplex/SeqCapEZ_Exome_v3.0_Design_Annotation_files/SeqCap_EZ_Exome_v3_hg19_capture_targets.bed'
ref_fa='/fdb/igenomes/Homo_sapiens/UCSC/hg19/Sequence/WholeGenomeFasta/genome.fa'

cnvkit.py batch BAM/*.bam -n \
    -t $exome_targets -f $ref_fa \
    --access /usr/local/apps/cnvkit/0.9.1/data/access-5k-mappable.hg19.bed \
    --output-reference my_flat_reference.cnn -d flat/ -p 30

I ended up running it in a terminal because the notebook wasn't showing error messages. Also, I had to install with pip a newer version of pysam (--user), because the version in the module was crashing cnvkit.

Before we go ahead with the call command, let's calculate some confidence intervals so we can use as filters in the future too:

In [ ]:
cd flat
while read s; do
    cnvkit.py call ${s}.cns -y -m threshold;
    cnvkit.py segmetrics -s ${s}.cn{s,r} --ci;
    cnvkit.py call ${s}.segmetrics.cns -y -m threshold --filter ci;
done < ../../conifer84/samples.txt

# TODO

* export VCFs?
* write PLINK files
* analyze data
* try other options for reference (see above)
* try with one of the aligners suggested in the CNVkit page